# LOFAR single station imaging - Video making

In [1]:
import numpy as np
import os
import re
import getpass
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2
import time
import sys
sys.path.append('..')

In [2]:
from lofarantpos.db import LofarAntennaDatabase
from lofarimaging import read_acm_cube, get_station_type, make_xst_plots

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
db = LofarAntennaDatabase()

In [11]:
username = getpass.getuser()
data_dir = f"/home/{username}/Documents/LV614_image_data/"
caltable_dir = f"/home/{username}/Documents/LV614_image_data/CalTables/"
caltable_dir = "CalTables/"

output_dir = "results/movie"
temp_dir = output_dir + "/temp"
fps = 10

# Observation that works
obs_dir = data_dir + "sess_sid20230111T071252_LV614/scan_59955.30061/LV614_20230111_071256_spw3_sb51_461_int1_dur24300_dir0,0,SUN_xst/"

# Observation info
station_name = "LV614"
integration_time_s = 2
rcu_mode = 3
height = 1.5
station_type = get_station_type(station_name)


In [6]:
def get_subbands(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    match = re.search(r'--xcsubband=(\d+)', content)
    return int(match.group(1)) if match else None

def get_obstime(file_path):
    obsdatestr, obstimestr, *_ = os.path.basename(file_path).rstrip(".dat").split("_")
    return datetime.datetime.strptime(obsdatestr + ":" + obstimestr, '%Y%m%d:%H%M%S')

def analyze_files(data_files):
    dat_files = sorted(glob.glob(os.path.join(data_files, '*.dat')))
    h_files = sorted(glob.glob(os.path.join(data_files, '*.h')))

    assert len(dat_files) == len(h_files), "Mismatch in number of .dat and .h files"

    data_list = []

    for dat_file, h_file in zip(dat_files, h_files):
        timestamp = get_obstime(dat_file)
        timestamp2 = get_obstime(h_file)
        if timestamp != timestamp2:
            print(f"Warning: timestamps do not match for {dat_file} and {h_file}")
        subband = get_subbands(h_file)

        data_list.append({
            "timestamp": timestamp,
            "subband": subband,
            "dat_file": dat_file,
            "h_file": h_file
        })

    df = pd.DataFrame(data_list)
    df = df.sort_values(by=["timestamp", "subband"]).reset_index(drop=True)

    average_measures_per_subband = round(df["subband"].value_counts().mean(), 2)
    measurement_duration = round((df["timestamp"].max() - df["timestamp"].min()).total_seconds() / len(df), 2) if len(df) > 1 else None

    summary = {
        "number_of_files": len(df),
        "subbands_available": {
            "first_subband": df["subband"].min(),
            "last_subband": df["subband"].max(),
            "total_subbands": len(df["subband"].dropna().unique())
        },
        "start_time": df["timestamp"].min(),
        "end_time": df["timestamp"].max(),
        "average_measures_per_subband": average_measures_per_subband,
        "measurement_duration": measurement_duration
    }

    return df, summary

def print_summary(summary):
    print("Summary of Analyzed Files:")
    print(f"Number of files: {summary['number_of_files']}")
    print(f"First and last subband: {summary['subbands_available']['first_subband']} - {summary['subbands_available']['last_subband']}")
    print(f"Total subbands: {summary['subbands_available']['total_subbands']}")
    print(f"Start time: {summary['start_time']}")
    print(f"End time: {summary['end_time']}")
    print(f"Average measurements per subband: {summary['average_measures_per_subband']}")
    print(f"Average measurement duration: {summary['measurement_duration']} seconds")


def generate_movie(sources, output_path, fps=10):
    if not sources:
        print("Image list is empty.")
        return

    # Use the first image to get the size of the frame
    image = cv2.imread(sources[0])
    if image is None:
        print(f"Error reading image: {sources[0]}. Aborting.")
        return

    height, width, _ = image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Iterate through the images and write them to the movie
    for path in sources:
        image = cv2.imread(path)
        if image is None:
            print(f"Error reading image: {path}. It will be skipped.")
            continue
        out.write(image)

    out.release()
    cv2.destroyAllWindows()


def generate_movie_from_list(list_path, output_path, fps=10):
    with open(list_path, "r") as file:
        sources = file.read().splitlines()
    generate_movie(sources, output_path, fps=fps)


In [ ]:
df, summary = analyze_files(obs_dir)
print_summary(summary)

Summary of Analyzed Files:
Number of files: 12150
First and last subband: 51 - 461
Total subbands: 411
Start time: 2023-01-11 07:12:56
End time: 2023-01-11 13:57:54
Average measurements per subband: 29.56
Average measurement duration: 2.0 seconds


,timestamp,subband,dat_file,h_file
0,2023-01-11 07:12:56,51,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
1,2023-01-11 07:12:58,52,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
2,2023-01-11 07:13:00,53,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
3,2023-01-11 07:13:02,54,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
4,2023-01-11 07:13:04,55,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
...,...,...,...,...
12145,2023-01-11 13:57:46,277,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
12146,2023-01-11 13:57:48,278,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
12147,2023-01-11 13:57:50,279,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...
12148,2023-01-11 13:57:52,280,/home/jan/Documents/LV614_image_data/sess_sid2...,/home/jan/Documents/LV614_image_data/sess_sid2...


In [ ]:
# Temporal evolution of the selected subbands

height = 1.5

subbands = [200, 280, 360]
#subbands = [255]

sky_movie = []
nf_movie = []

for subband in subbands:
    subband_data = df[df['subband'] == subband]#.head(3)

    for index, row in subband_data.iterrows():
        xst_filename = row['dat_file']
        obstime = row['timestamp']
        time.sleep(0.1)

        try:
            print(f"Generating image for subband {subband} at time {obstime} and heigth {height} m.")
            visibilities = read_acm_cube(xst_filename, station_type)[0]
            sky_image_path, nf_image_path, _ = make_xst_plots(visibilities, station_name, obstime, subband, rcu_mode, map_zoom=18, outputpath=temp_dir, mark_max_power=True, height=height, return_only_paths=True)
            sky_movie.append(sky_image_path)
            nf_movie.append(nf_image_path)
        except Exception as e:
            print(f"Error generating image for {xst_filename}: {e}")

# Export the image lists for movie generation
with open(f"{temp_dir}/sky_image_list_time_sweep.txt", "w") as sky_file:
    sky_file.write("\n".join(sky_movie))

with open(f"{temp_dir}/nf_image_list_time_sweep.txt", "w") as nf_file:
    nf_file.write("\n".join(nf_movie))

print("Image generation complete for time sweep.")

generate_movie_from_list(f"{temp_dir}/sky_image_list_time_sweep.txt", f"{output_dir}/sky_movie_time_sweep.mp4", fps=fps)
generate_movie_from_list(f"{temp_dir}/nf_image_list_time_sweep.txt", f"{output_dir}/nf_movie_time_sweep.mp4", fps=fps)

print("Movie generation complete.")

Generating image for subband 200 at time 2023-01-11 07:17:54 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 07:31:36 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 07:45:18 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 07:59:00 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 08:26:24 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 08:40:06 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 08:53:48 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 09:07:30 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 09:21:12 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 09:34:54 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 09:48:36 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 

In [ ]:
# Subband sweep at a given time

height = 1.5

subband_min = 180
subband_max = 460
subband_step = 2
subbands = df["subband"].unique()
subbands = [s for s in sorted(subbands) if subband_min <= s <= subband_max][::subband_step]

min_time = df["timestamp"].min()
times = [
    #min_time + datetime.timedelta(minutes=5),
    min_time + datetime.timedelta(minutes=50),
    min_time + datetime.timedelta(minutes=200)
]

sky_movie = []
nf_movie = []

for t in times:

    for subband in subbands:
        subband_data = df[df['subband'] == subband]

        subband_data = subband_data[subband_data['timestamp'] >= t]
        subband_data = subband_data.sort_values('timestamp')
        closest_row = subband_data.iloc[0] if not subband_data.empty else None

        if closest_row is None:
            print(f"No data found for subband {subband} at {t}")
            continue

        xst_filename = closest_row['dat_file']
        obstime = closest_row['timestamp']

        try:
            print(f"Generating image for subband {subband} at time {obstime} and heigth {height} m.")
            visibilities = read_acm_cube(xst_filename, station_type)[0]
            sky_image_path, nf_image_path, _ = make_xst_plots(visibilities, station_name, obstime, subband, rcu_mode, map_zoom=18, outputpath=temp_dir, mark_max_power=True, height=height, return_only_paths=True)
            sky_movie.append(sky_image_path)
            nf_movie.append(nf_image_path)
        except Exception as e:
            print(f"Error generating image for {xst_filename}: {e}")

# Export the image lists for movie generation
with open(f"{temp_dir}/sky_image_list_subband_sweep.txt", "w") as sky_file:
    sky_file.write("\n".join(sky_movie))

with open(f"{temp_dir}/nf_image_list_subband_sweep.txt", "w") as nf_file:
    nf_file.write("\n".join(nf_movie))

print("Image generation complete for subband sweep.")

generate_movie_from_list(f"{temp_dir}/sky_image_list_subband_sweep.txt", f"{output_dir}/sky_movie_subband_sweep.mp4", fps=fps)
generate_movie_from_list(f"{temp_dir}/nf_image_list_subband_sweep.txt", f"{output_dir}/nf_movie_subband_sweep.mp4", fps=fps)

print("Movie generation complete.")

Generating image for subband 180 at time 2023-01-11 08:12:02 and heigth 1.5 m.
Generating image for subband 182 at time 2023-01-11 08:12:06 and heigth 1.5 m.
Generating image for subband 184 at time 2023-01-11 08:12:10 and heigth 1.5 m.
Generating image for subband 186 at time 2023-01-11 08:12:14 and heigth 1.5 m.
Generating image for subband 188 at time 2023-01-11 08:12:18 and heigth 1.5 m.
Generating image for subband 190 at time 2023-01-11 08:12:22 and heigth 1.5 m.
Generating image for subband 192 at time 2023-01-11 08:12:26 and heigth 1.5 m.
Generating image for subband 194 at time 2023-01-11 08:12:30 and heigth 1.5 m.
Generating image for subband 196 at time 2023-01-11 08:12:34 and heigth 1.5 m.
Generating image for subband 198 at time 2023-01-11 08:12:38 and heigth 1.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 1.5 m.
Generating image for subband 202 at time 2023-01-11 08:12:46 and heigth 1.5 m.
Generating image for subband 204 at time 2023-01-11 

In [ ]:
# Height sweep at a given time and subband

height_min = 1
height_max = 151
height_step = 2.5

subbands = [200, 280, 360]
#subbands = [255]

min_time = df["timestamp"].min()
times = [
    #min_time + datetime.timedelta(minutes=5),
    min_time + datetime.timedelta(minutes=50),
    min_time + datetime.timedelta(minutes=200)
]

sky_movie = []
nf_movie = []

for t in times:
    for subband in subbands:
        subband_data = df[df['subband'] == subband]

        subband_data = subband_data[subband_data['timestamp'] >= t]
        subband_data = subband_data.sort_values('timestamp')
        closest_row = subband_data.iloc[0] if not subband_data.empty else None

        if closest_row is None:
            print(f"No data found for subband {subband} at {t}")
            continue

        for height in np.arange(height_min, height_max, height_step):
            xst_filename = closest_row['dat_file']
            obstime = closest_row['timestamp']

            try:
                print(f"Generating image for subband {subband} at time {obstime} and heigth {height} m.")
                visibilities = read_acm_cube(xst_filename, station_type)[0]
                sky_image_path, nf_image_path, _ = make_xst_plots(visibilities, station_name, obstime, subband, rcu_mode, map_zoom=18, outputpath=temp_dir, mark_max_power=True, height=height, return_only_paths=True)
                sky_movie.append(sky_image_path)
                nf_movie.append(nf_image_path)
            except Exception as e:
                print(f"Error generating image for {xst_filename}: {e}")

with open(f"{temp_dir}/nf_image_list_heigth_sweep.txt", "w") as nf_file:
    nf_file.write("\n".join(nf_movie))

print("Image generation complete for height sweep.")

generate_movie_from_list(f"{temp_dir}/nf_image_list_heigth_sweep.txt", f"{output_dir}/nf_movie_heigth_sweep.mp4", fps=fps)

print("Movie generation complete.")

Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 1.0 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 3.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 6.0 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 8.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 11.0 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 13.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 16.0 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 18.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 21.0 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 23.5 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 26.0 m.
Generating image for subband 200 at time 2023-01-11 08:12:42 and heigth 28.5 m.
Generating image for subband 200 at time 202

In [12]:

generate_movie_from_list(f"{temp_dir}/sky_image_list_time_sweep.txt", f"{output_dir}/sky_movie_time_sweep.mp4", fps=fps)
generate_movie_from_list(f"{temp_dir}/nf_image_list_time_sweep.txt", f"{output_dir}/nf_movie_time_sweep.mp4", fps=fps)

generate_movie_from_list(f"{temp_dir}/sky_image_list_subband_sweep.txt", f"{output_dir}/sky_movie_subband_sweep.mp4", fps=fps)
generate_movie_from_list(f"{temp_dir}/nf_image_list_subband_sweep.txt", f"{output_dir}/nf_movie_subband_sweep.mp4", fps=fps)

generate_movie_from_list(f"{temp_dir}/nf_image_list_heigth_sweep.txt", f"{output_dir}/nf_movie_heigth_sweep.mp4", fps=fps)

print("Movie generation complete.")

Movie generation complete.
